# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Machine Learning** </center>
---

**Date**: November, 2025

**Student Name**: Luis Angel Santana Hernandez

**Professor**: Pablo Camarillo Ramirez

## Introduction

In this this projectwe are going predict traffic conditions using machine learning on data collected through a computer vision system. The dataset includes counts of different vehicle types, along with time related data such as time, date, and day of the week. the data is labeled with a traffic category: 1-Heavy, 2-High, 3-Normal, or 4-Low. Because the target variable consists classes representing traffic levels, the problem is classification related.

Among the available machine learning algoriths, the classification category is the usefull, since it allows the model to learn from examples and give an use this characteristics to lable with traffic classes. Within the classification algorithms, this project wll use Random Forest Classifier, chosen for its performance, robustnes to noise and ability to handle both numerical and categorical data

# Create SparkSession

In [130]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Proyect ML: Random Forest") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Model Characterstics

Page :https://www.kaggle.com/datasets/hasibullahaman/traffic-prediction-dataset

size: 393.11 kB

files:2



Data present:


1. Date

The calendar date of the observation.

Format: 1-31

Used to analyze trends across days of the month.

2. Time

The hour and minutes when the data was recorded.

Data is logged every 15 minutes.

Example: 14:30

3. Day of the week

Example values: "Monday", "Tuesday", etc.

4. CarCount

Number of cars detected by the computer in that 15-minute interval.

5. BikeCount

Number of bikes/motorcycles detected in that interval.

6. BusCount

Number of buses detected.

7. TruckCount

Number of trucks detected.

8. Total

The sum of all detected vehicle types (Car + Bike + Bus + Truck).

Represents the overall traffic volume.

9. Traffic Situation / Traffic

label used for classification.

Describes traffic level.

Encoded as:

1 = Heavy

2 = High

3 = Normal

4 = Low

This is the column we will predict using Random Forest Classification.

In [131]:
from luis_santana.spark_utils import SparkUtils

#import 
trafic_schema = SparkUtils.generate_schema([
    ("Time", "string"), 
    ("Date", "int"), 
    ("Day of the week", "string"), 
    ("CarCount", "int"), 
    ("BikeCount", "int"), 
    ("BusCount", "int"),
    ("TruckCount", "int"), 
    ("Total", "int"), 
    ("Trafic Situation", "string")])

trafic_df = spark.read \
                .option("header", "true") \
                .schema(trafic_schema) \
                .csv("/opt/spark/work-dir/data/Proyect/ml/")

#eliminate nulls
trafic_df = trafic_df.na.drop()
# trafic_df.count()

number of elements: 8928

In [132]:
# balance
# counts = trafic_df.groupBy("Trafic Situation").count()
# counts.show()

```python
 +----------------+-----+
|Trafic Situation|count|
+----------------+-----+
|            high|  692|
|             low| 1138|
|           heavy| 1819|
|          normal| 5279|
+----------------+-----
```

This is an unbalanced model since there is more normal trafic than the other values

## Convert string labels to numeric


In [133]:
# trafic_df.show()

In [134]:
# convert 12 hour time to float time
from pyspark.sql.functions import to_timestamp,hour,minute,col

trafic_df = trafic_df.withColumn("HourFloat", 
    hour(to_timestamp(col("Time"), "h:mm:ss a"))
    + minute(to_timestamp(col("Time"), "h:mm:ss a"))/60.0
)


# trafic_df.show()

In [135]:
#transfor day of the week to numeric(normally)

# from pyspark.ml.feature import StringIndexer
# day_indexer = StringIndexer(inputCol="Day of the week", outputCol="DayIndex")
# trafic_df = day_indexer.fit(trafic_df).transform(trafic_df)
# trafic_df.show()

# I want monday to be 0 and sunday to be 6
from pyspark.sql.functions import col, when

trafic_df = trafic_df.withColumn(
    "DayIndex",
    when(col("Day of the week") == "Monday",    1)
    .when(col("Day of the week") == "Tuesday",  2)
    .when(col("Day of the week") == "Wednesday",3)
    .when(col("Day of the week") == "Thursday", 4)
    .when(col("Day of the week") == "Friday",   5)
    .when(col("Day of the week") == "Saturday", 6)
    .when(col("Day of the week") == "Sunday",   7)
)
# trafic_df.show()

In [136]:
#transform trafic situation to numeric (noramlly)
## Convert string labels to numeric
# from pyspark.ml.feature import StringIndexer
# label_indexer = StringIndexer(inputCol="Trafic Situation", outputCol="label")
# trafic_df = label_indexer.fit(trafic_df).transform(trafic_df)
# trafic_df.show()

# i want high = 0, low = 1, heavy = 2, normal = 3
trafic_df = trafic_df.withColumn(
    "label",
    when(col("Trafic Situation") == "high",   0)
    .when(col("Trafic Situation") == "low",    1)
    .when(col("Trafic Situation") == "heavy",  2)
    .when(col("Trafic Situation") == "normal", 3)
)
# trafic_df.show()

## Asemble and spliting

In [137]:
feature_cols = ["CarCount", "BikeCount", "BusCount", "TruckCount", "Total", "HourFloat", "DayIndex","Date"]
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
assembled_df = assembler.transform(trafic_df)
assembled_df = assembled_df.select("features", "label")
# assembled_df.show()

In [138]:
#plit data
train_df, test_df = assembled_df.randomSplit([0.8, 0.2], seed=42)

## Create and train Ml

matadata numbers found by empirical methot

In [139]:
from pyspark.ml.classification import RandomForestClassifier

# Define the Random Forest model
rf_trafic = RandomForestClassifier(
    labelCol="label",
    featuresCol="features",
    numTrees=110,
    maxDepth=10,
    seed=42
)

# Build a pipeline

rf_trafic = rf_trafic.fit(train_df)

# Save the model
rf_path = "/opt/spark/work-dir/data/Proyect/mlmodel_110/"
rf_trafic.write().overwrite().save(rf_path)
# print(f"Random forest model generated: {rf_trafic.toDebugString}")

25/11/22 00:14:12 WARN DAGScheduler: Broadcasting large task binary with size 1007.8 KiB
25/11/22 00:14:13 WARN DAGScheduler: Broadcasting large task binary with size 1621.7 KiB
25/11/22 00:14:14 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
25/11/22 00:14:16 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB
25/11/22 00:14:17 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
25/11/22 00:14:23 WARN TaskSetManager: Stage 31 contains a task of very large size (1627 KiB). The maximum recommended task size is 1000 KiB.


## ML Evaluation

using f1 score since this data is not balanced

In [140]:
# Read the RF model
from pyspark.ml.classification import RandomForestClassificationModel
rf_path = "/opt/spark/work-dir/data/Proyect/mlmodel_110/"

rf_model_saved = RandomForestClassificationModel.load(rf_path)

In [141]:
# evaluate
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


predictions_rf = rf_model_saved.transform(test_df)

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

f1 = evaluator.evaluate(predictions_rf,
                {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")  

# trees=50 F1 Score: 0.9247594289588481
# trees=100 F1 Score: 0.9251138445634184
# trees=110, Depht=5 F1 Score: 0.9291349287137586
# trees=110, Depht=10 F1 Score: 0.9299846492501319
# trees=110, Depht=20 F1 Score: 0.9095975809451398
# trees=125 F1 Score: 0.9268303004137959
# trees=150 F1 Score: 0.9245284793669771

# presition = evaluator.evaluate(predictions_rf,
#                 {evaluator.metricName: "weightedPrecision"})
# print(f"Presition: {presition}")

# recall_rf = evaluator.evaluate(predictions_rf,
#                 {evaluator.metricName: "weightedRecall"})
# print(f"Recall of Random Forest: {recall_rf}")

25/11/22 00:14:34 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


F1 Score: 0.9299846492501319


In [142]:
sc.stop()